In [37]:
import pandas as pd
from faker import Faker
import random
import warnings

warnings.filterwarnings('ignore')

First of all we need to genarete some random data to work with. The Faker library can generate names, addresses, dates, and other types of random data. The Faker library is especially useful for creating large amounts of test data that is random but reproducible.



In [30]:
# Initialize Faker
fake = Faker()

# Create an empty data variable and populate with synthetic data
data = []

for _ in range(random.randint(1000, 2000)):
    data.append({
        'Name':fake.name(),
        'Age':random.randint(18,70),
        'Email':fake.email(),
        'Phone':fake.phone_number(),
        'Address':fake.address(),
        'Salary':random.randint(20000, 150000),
        'Join_Date':fake.date_this_century(),
        'Employment_Status':random.choice(['Full-Time', 'Part-Time', 'Contract']),
        'Department':random.choice(['IT', 'Engineering', 'Finance', 'HR', 'Marketing', 'Customer Success'])
    })

print(f'Generated data has {str(len(data))} records.')

Generated data has 1755 records.


Define a function that will help defining the limits of some functions from the random library.

In [7]:
def gen_limit(data:list , percent: float):
    limit = round(len(data) * percent)
    return limit

Let's skew the data in order to simulate a real world case where many times the data that comes with some missing values.

The values used in the ```gen_limit``` function were defined just for the purpose of demonstration. Feel free to experiment with different values as you wish.

In [9]:
# Adding missing values to the Email column
for i in random.sample(range(len(data)), random.randint(gen_limit(data, 0.1), gen_limit(data, 0.15))):
    data[i]['Email'] = None

# Adding missing values to the Phone column
for i in random.sample(range(len(data)), random.randint(gen_limit(data, 0.15), gen_limit(data, 0.2))):
    data[i]['Phone'] = None

# Adding missing values to the Address column
for i in random.sample(range(len(data)), random.randint(gen_limit(data, 0.05), gen_limit(data, 0.15))):
    data[i]['Address'] = None

Next let's duplicate a couple of rows since in real cases is really comun for this to happen. 

In [12]:
data.extend(random.sample(data, random.randint(gen_limit(data, 0.1), gen_limit(data, 0.15))))

Now add some outliers so we can deal with it later. 

In [13]:
# Add outliers to the Salary column
for i in random.sample(range(len(data)), random.randint(gen_limit(data, 0.01), gen_limit(data, 0.02))):
    data[i]['Salary'] = random.randint(300000, 700000)

Finally, create a Dataframe using the data that has been created so far and check how it looks.

In [35]:
df = pd.DataFrame(data)
df.head()

,Name,Age,Email,Phone,Address,Salary,Join_Date,Employment_Status,Department
0,Rebecca Navarro,65,sherry19@example.com,(302)368-1704x731,"7048 Ellis Freeway\nSouth Adamview, AZ 39675",29202,2010-12-20,Contract,HR
1,Sandra Hogan,56,mrogers@example.com,358-343-3256,"432 Robert Avenue Suite 887\nSchneiderburgh, N...",141492,2017-11-07,Part-Time,Finance
2,Robert Navarro,28,allenkelly@example.org,(413)941-2437x756,"321 Kelly Vista Suite 492\nLake William, OR 33699",63645,2010-04-04,Full-Time,HR
3,Ricky Mendoza,19,jfrancis@example.com,+1-203-378-6348x646,"393 Ryan Stream Apt. 616\nWest Sherri, UT 38655",105629,2019-09-16,Contract,HR
4,Christopher Baker,51,rachelpace@example.net,851.267.8159,"138 Allison Street\nLake Michelle, IN 32468",68551,2005-02-02,Full-Time,IT


Use the ```info()``` method to check how the amount of values of the columns that we skew differ from the rest.

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1318 entries, 0 to 1317
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               1318 non-null   object
 1   Age                1318 non-null   int64 
 2   Email              1183 non-null   object
 3   Phone              1107 non-null   object
 4   Address            1230 non-null   object
 5   Salary             1318 non-null   int64 
 6   Join_Date          1318 non-null   object
 7   Employment_Status  1318 non-null   object
 8   Department         1318 non-null   object
dtypes: int64(2), object(7)
memory usage: 92.8+ KB


Now let starting dealing with the duplicate records.

Duplicate records are frequent in the real data cleaning cases and can often skew the data analysis.

In [18]:
duplicates = df.duplicated().sum()
print(f'Number of duplicate rows: {duplicates}')

df.drop_duplicates(inplace=True)

Number of duplicate rows: 121


Next we'll fill the empty values in order to have a string to inform that the values that are missing are unknow since the person related to could possibly not have informed them.

In [19]:
# Check the amount of missing values in each column
missing_values = df.isna().sum()
print('Number of missing values:')
print(missing_values)

Number of missing values:
Name                   0
Age                    0
Email                124
Phone                186
Address               82
Salary                 0
Join_Date              0
Employment_Status      0
Department             0
dtype: int64


In [38]:
# Fill the missing values of each column with more intuitive info
df['Email'].fillna('unknown', inplace=True)
df['Phone'].fillna('unknown', inplace=True)
df['Address'].fillna('unknown', inplace=True)

Another crucial detail that can mess with the analysis are data types.

It's really important to check wether the columns data types are properly set so it will work accordingly to your analysis later.

In our case the colun Join_Date is not formatted as date, so we'll fix this now.

In [22]:
# Apply the correct data type to the column using pandas to_datetime method
df['Join_Date'] = pd.to_datetime(df['Join_Date'])
print('Join_Date column after conversion:')
print(df['Join_Date'].head())

Join_Date column after conversion:
0   2001-06-23
1   2007-10-17
2   2015-12-04
3   2008-03-12
4   2022-10-05
Name: Join_Date, dtype: datetime64[ns]


Another commom practice that can help during the process of data cleaning is to create new useful columns.

For example we could create a columns that tells the amount of years each person in the dataframe has worked, ubtracting the current year from the Join_Date year.

In [23]:
df['Years_Employed'] = pd.Timestamp.now().year - df['Join_Date'].dt.year
print('New Column "Years_Employed":')
print(df[['Join_Date', 'Years_Employed']].head())

New Column "Years_Employed":
   Join_Date  Years_Employed
0 2001-06-23              23
1 2007-10-17              17
2 2015-12-04               9
3 2008-03-12              16
4 2022-10-05               2


It’s quite common to run into string fields with inconsistent formatting or similar issues.

Cleaning text can be as simple as applying a case conversion or as hard as writing a complex regular expression to get the string to the required format.

In our example, the random genareted addresses can contain many '\n' characters than hinder redability, so we can remove them using the ```replace``` method.

In [24]:
df['Address'] = df['Address'].str.replace('\n', ' ', regex=False)
print('Adress after text cleaning:')
print(df['Address'].head())

Adress after text cleaning:
0    79402 Peterson Drives Apt. 511 Davisstad, PA 3...
1    55341 Amanda Gardens Apt. 764 Lake Mark, WI 07832
2                710 Eric Estate Carlsonfurt, MS 78605
3                809 Burns Creek Natashaport, IA 08093
4    33150 Brianna Avenue Apt. 031 Port Markhaven, ...
Name: Address, dtype: object


We still have the problem of outliers in our data, since previously salaries were randomly generated far above the others.

Outliers can also be seen frequently and can make us draw distorted conclusions from the data, so it is always a good idea to check if they exist and avoid them.

Let's check which salaries to find out wich values are more than three standard deviation away from the mean.

In [25]:
z_scores = (df['Salary'] - df['Salary'].mean()) / df['Salary'].std()
outliers = df[abs(z_scores) > 3]
print('Outliers based on salary:')
print(df[['Name', 'Salary']].head())

Outliers based on salary:
              Name  Salary
0     Allison Hill  149966
1  Kimberly Dudley   40179
2    Renee Morales   51062
3  Tricia Valencia  114261
4   Theresa Miller   51131


Finally, take a look in the new cleaned and transformed data, where the data is correctly formatted and there are no more missing values in the columns or outliers.

In [26]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1197 entries, 0 to 1196
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               1197 non-null   object        
 1   Age                1197 non-null   int64         
 2   Email              1197 non-null   object        
 3   Phone              1197 non-null   object        
 4   Address            1197 non-null   object        
 5   Salary             1197 non-null   int64         
 6   Join_Date          1197 non-null   datetime64[ns]
 7   Employment_Status  1197 non-null   object        
 8   Department         1197 non-null   object        
 9   Years_Employed     1197 non-null   int32         
dtypes: datetime64[ns](1), int32(1), int64(2), object(6)
memory usage: 98.2+ KB
None


In [39]:
df.head()

,Name,Age,Email,Phone,Address,Salary,Join_Date,Employment_Status,Department
0,Rebecca Navarro,65,sherry19@example.com,(302)368-1704x731,"7048 Ellis Freeway\nSouth Adamview, AZ 39675",29202,2010-12-20,Contract,HR
1,Sandra Hogan,56,mrogers@example.com,358-343-3256,"432 Robert Avenue Suite 887\nSchneiderburgh, N...",141492,2017-11-07,Part-Time,Finance
2,Robert Navarro,28,allenkelly@example.org,(413)941-2437x756,"321 Kelly Vista Suite 492\nLake William, OR 33699",63645,2010-04-04,Full-Time,HR
3,Ricky Mendoza,19,jfrancis@example.com,+1-203-378-6348x646,"393 Ryan Stream Apt. 616\nWest Sherri, UT 38655",105629,2019-09-16,Contract,HR
4,Christopher Baker,51,rachelpace@example.net,851.267.8159,"138 Allison Street\nLake Michelle, IN 32468",68551,2005-02-02,Full-Time,IT


Being comfortable with the steps above is crucial to a data professional since cases similiar to these are extremely frequent, and our mission is to deliver good consistent and reliable data, wheter we are storing in data warehouses and data lakes, analysing to create a report or creating a new dashboard to an important client.